In [2]:
import numpy as np
import pandas as pd
import requests
from pandas.io.json import json_normalize
import folium
print('Libraries imported')

Libraries imported


#### Extract the content from the link and specify to extract the first item (table) and save it as you want

In [3]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = requests.get(url).content
df_list = pd.read_html(html)
df = df_list[0]
print(df)
df.to_csv('toronto_data.csv')

    Postal Code           Borough  \
0           M1A      Not assigned   
1           M2A      Not assigned   
2           M3A        North York   
3           M4A        North York   
4           M5A  Downtown Toronto   
..          ...               ...   
175         M5Z      Not assigned   
176         M6Z      Not assigned   
177         M7Z      Not assigned   
178         M8Z         Etobicoke   
179         M9Z      Not assigned   

                                         Neighbourhood  
0                                         Not assigned  
1                                         Not assigned  
2                                            Parkwoods  
3                                     Victoria Village  
4                            Regent Park, Harbourfront  
..                                                 ...  
175                                       Not assigned  
176                                       Not assigned  
177                                       

#### Read and show csv file

In [4]:
df_toronto = pd.read_csv('toronto_data.csv')

In [5]:
df_toronto.head()

,Unnamed: 0,Postal Code,Borough,Neighbourhood
0,0,M1A,Not assigned,Not assigned
1,1,M2A,Not assigned,Not assigned
2,2,M3A,North York,Parkwoods
3,3,M4A,North York,Victoria Village
4,4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Drop Unnamed: 0 column and clear all rows which are assigned a "Not Assigned" value in Borough column

In [6]:
df_toronto = df_toronto.drop(['Unnamed: 0'], axis=1)

In [7]:
df_toronto = df_toronto[df_toronto.Borough != 'Not assigned']

In [8]:
df_toronto = df_toronto.reset_index()

In [9]:
df_toronto = df_toronto.drop(['index'], axis=1)

In [13]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Import Geocoder, define a function called get_geocoder creating a loop to retrieve all coordinates related to Toronto area postal codes and assig coordinates to each row by postal code to the dataframe

In [32]:
import geocoder

def get_geocoder(postal_code):
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g=geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

for i in range (0,len(df_toronto['Postal Code'])):
    df_toronto.iloc[i]['Latitude'],df_toronto.iloc[i]['Longitude']=get_geocoder(df_toronto.iloc[i]['Postal Code'])

In [33]:
df_toronto['Latitude'], df_toronto['Longitude'] = zip(*df_toronto['Postal Code'].apply(get_geocoder))

In [34]:
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
